# Distributed data parallel MNIST training with PyTorch and SMDataParallel


## Background
SMDataParallel is a new capability in Amazon SageMaker to train deep learning models faster and cheaper. SMDataParallel is a distributed data parallel training framework for PyTorch. 

This notebook example shows how to use SMDataParallel with PyTorch in SageMaker using MNIST dataset.

For more information:
1. [PyTorch in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)
2. [SMDataParallel PyTorch API Specification] < LINK TO BE ADDED >
3. [Getting started with SMDataParallel on SageMaker] < LINK TO BE ADDED >

**NOTE:** This example requires SageMaker Python SDK v2.X.


### Dataset
This example uses the MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).



### SageMaker execution roles

The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with the appropriate full IAM role arn string(s).

In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## Model training with SMDataParallel

### Training script

The MNIST dataset is downloaded using the `torchvision.datasets` PyTorch module; you can see how this is implemented in the `train_pytorch_smdataparallel_mnist.py` training script that is printed out in the next cell.

The training script provides the code you need for distributed data parallel (DDP) training using SMDataParallel. The training script is very similar to a PyTorch training script you might run outside of SageMaker, but modified to run with SMDataParallel. SMDataParallel's PyTorch client provides an alternative to PyTorch's native DDP. For details about how to use SMDataParallel's DDP in your native PyTorch script, see the Getting Started with SMDataParallel tutorials.


In [2]:
!pygmentize code/train_pytorch_smdataparallel_mnist.py

# Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

from __future__ import print_function

import os
import argparse
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Network definition
from model_def import Net

# Import SMDataParallel PyTorch Modules
from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataPar

### Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [3]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(base_job_name='pytorch-smdataparallel-mnist',
                        source_dir='code',
                        entry_point='train_pytorch_smdataparallel_mnist.py',
                        role=role,
                        framework_version='1.6.0',
                        py_version='py3',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                                 }
                                          }
                                      },
                        debugger_hook_config=False)

In [4]:
estimator.fit()

2020-12-10 20:14:55 Starting - Starting the training job...
2020-12-10 20:14:58 Starting - Launching requested ML instances.........
2020-12-10 20:16:45 Starting - Preparing the instances for training.........
2020-12-10 20:18:26 Downloading - Downloading input data
2020-12-10 20:18:26 Training - Downloading the training image.........
2020-12-10 20:19:56 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-10 20:19:47,428 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-12-10 20:19:47,505 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-10 20:19:53,768 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-10 20:19:51,055 sagemaker-training-


2020-12-10 20:20:04 Failed - Training job failed


UnexpectedStatusException: Error for Training job pytorch-smdataparallel-mnist-2020-12-10-20-14-54-846: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python train_pytorch_smdataparallel_mnist.py"
Traceback (most recent call last):
  File "train_pytorch_smdataparallel_mnist.py", line 30, in <module>
    from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
ModuleNotFoundError: No module named 'smdistributed'

## Next steps

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [5]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

Storing s3://sagemaker-us-east-1-835319576252/pytorch-smdataparallel-mnist-2020-12-10-20-14-54-846/output/model.tar.gz as model_data
Stored 'model_data' (str)
